In [4]:
#!pip install tqdm

In [5]:
import ollama
import time
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from numba import jit
from tqdm import tqdm

In [15]:
#df = pd.read_csv("patent-1k-clean.csv")
df = pd.read_csv("patent-1m-clean.csv")

df.head()

# Models -> all-minilm (384), nomic-embed-text (768), mxbai-embed-large (1024)

,patent_id,patent_date,patent_title,patent_abstract
0,10000000,6/19/2018,Coherent LADAR using intra-pixel quadrature de...,A frequency modulated (coherent) laser detecti...
1,10000001,6/19/2018,Injection molding machine and mold thickness c...,The injection molding machine includes a fixed...
2,10000002,6/19/2018,Method for manufacturing polymer film and co-e...,The present invention relates to: a method for...
3,10000003,6/19/2018,Method for producing a container from a thermo...,The invention relates to a method for producin...
4,10000004,6/19/2018,"Process of obtaining a double-oriented film, c...",The present invention relates to provides a do...


In [16]:
# Read embeddings
# 1k_min_title.npy, 1k_medium_title.npy, 1k_large_title.npy, 1k_min_abstract.npy, 1k_medium_abstract.npy, 1k_large_abstract.npy
#title_dict = np.load('1k_min_title.npy', allow_pickle=True)
#title_dict = np.load('1k_large_title.npy', allow_pickle=True)
#title_dict = np.load('1m_min_title.npy', allow_pickle=True)
#title_dict = np.load('1m_max_title.npy', allow_pickle=True)
#title_dict = np.load('1k_large_abstract.npy', allow_pickle=True)
title_dict = np.load('1m_large_abstract.npy', allow_pickle=True)

In [17]:
type(title_dict)

numpy.ndarray

In [8]:
#print(title_dict)
#list(title_dict)
xyz = title_dict.tolist()
#type(xyz)
#xyz[10000000]
keys = xyz.keys()
#len(keys)
#keys
# d = dict(enumerate(title_dict.flatten(), 1))
# len(d)
#df['patent_id'].loc(10000036)
rrr = df.loc[df['patent_id'] == 10000036]
print(rrr['patent_title'].values[0])
#print(rrr)

High kinetic energy penetrator shielding and high wear resistance materials fabricated with boron nitride nanotubes (BNNTS) and BNNT polymer composites


In [30]:
# for index, row in df.iterrows():
#      #print(row["c1"], row["c2"])
#     print(row["patent_id"])

# for row in df.itertuples(index=False):
#     print(row[df.columns.get_loc('patent_id')])

# for index, row in smaller_df.iterrows():
#     print(row["patent_id"])

# for row in smaller_df.itertuples(index=False):
#     print(row[smaller_df.columns.get_loc('patent_id')])

t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

19:47:34


In [6]:
#Get Embedding for My Patent Title
my_patent_title = 'Switch With Diagnostic Capability'
my_patent_title_response = ollama.embeddings(model="all-minilm", prompt=my_patent_title)
#my_patent_title_response = ollama.embeddings(model="mxbai-embed-large", prompt=my_patent_title)
info = list(my_patent_title_response.values())[0]
#patent_ids = 
len(info)
info_arr = np.array(info)
#len(my_patent_title_response)

In [7]:
size = 384
default_value = 0.5

# Create a list with the specified size filled with a default value using a for loop
my_list = []
for _ in range(size):
    my_list.append(default_value)

arr = np.array(my_list)

In [10]:
# Compare My Patent Title Embedding to the other embeddings and find the closest title
# I need to track PatentID of closest title, and then find the title from the CSV file
# Cosine Simularity

def cosine_sim(vec1, vec2):
    """
    Calculates the cosine similarity between two vectors.
    """
    return cosine_similarity([vec1], [vec2])[0][0]

# # Example vectors
# embedding1 = np.array([0.2, 0.5, 0.8])
# embedding2 = np.array([0.1, 0.6, 0.7])

# # Calculate cosine similarity
# similarity = cosine_sim(embedding1, embedding2)

start = time.time()
similarity = cosine_sim(arr, info_arr)
end = time.time()
dtime = "Execution time = " + str(end - start)
print(dtime)

print("Cosine Similarity:", similarity)

NameError: name 'arr' is not defined

In [11]:
# Try to Speed up Simularity with Numba
@jit(nopython=True)
def cosine_similarity_numba(a, b):
    dot = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot / (norm_a * norm_b)

In [22]:
len(patents_dict)

999

In [18]:
#Get Embedding for My Patent Title
#my_patent_title = 'Switch With Diagnostic Capability'
my_patent_title = 'A switch is provided with the ability to diagnose its own operation and provide signals that represent the actuation of \
the switch and various diagnostic outputs that indicate fault conditions or potential fault conditions with regard to either \
the switch or related machinery. A magnetically sensitive component, such as a Hall element, is used to provide an \
analog output signal that is representative of the position of an actuator to which a magnet structure is rigidly attached. \
As the actuator moves in response to an external force, the magnet structure moves with respect to a stationary magnetically sensitive \
component. The analog output from the magnetically sensitive component can be used to determine the position of the switch. Based on information, \
a microprocessor provides a binary output signal indicating whether the switch is actuated or deactuated. In addition, the \
movement of the actuator can be compared to various thresholds to determine the operate point, the overtravel point, the total travel and the velocity of the switch plunger.'
#my_patent_title_response = ollama.embeddings(model="all-minilm", prompt=my_patent_title)
my_patent_title_response = ollama.embeddings(model="mxbai-embed-large", prompt=my_patent_title)
info = list(my_patent_title_response.values())[0]
my_patent_title_arr = np.array(info)

patents_dict = title_dict.tolist()

max_simularity = -10.0
max_key = -10

#print("Length of patents diction = " + len(patents_dict.items()))
#print("Length of patents dictionary = " + str(len(patents_dict)))

start = time.time()

my_counter = 0

#for item in patents_dict.items():
#for key, value in patents_dict.items():
for key, value in tqdm (patents_dict.items()):
    # if (my_counter % 100000) == 0:
    #     print(my_counter)
    #     t = time.localtime()
    #     current_time = time.strftime("%H:%M:%S", t)
    #     print(current_time)
    my_counter += 1
    #similarity = cosine_sim(my_patent_title_arr, value)
    similarity = cosine_similarity_numba(my_patent_title_arr, value)
    if similarity > max_simularity:
        max_simularity = similarity
        max_key = key

print("Max Simularity = " + str(max_simularity))
print("Patent ID with Max Simularity = " + str(max_key))
rrr = df.loc[df['patent_id'] == max_key]
#print("Most simular patent, by title is -> " + rrr['patent_title'].values[0])
print("Most simular patent, by abstract is -> " + rrr['patent_abstract'].values[0])
end = time.time()
dtime = "Execution time = " + str(end - start)
print(dtime)

100%|████████████████████████████████████████████████████████████████████| 1048575/1048575 [00:05<00:00, 204213.64it/s]

Max Simularity = 0.827431031751498
Patent ID with Max Simularity = 10502806
Most simular patent, by abstract is -> Devices and methods are provided where switches associated with a magnetic field sensor are used to provide error information. In particular, a device is provided that includes a magnetic field sensor, a plurality of switches associated with the magnetic field sensor, and a control circuit configured to control the plurality of switches and to provide at least one signal indicative of a fault based on operation of the switches.
Execution time = 5.138123989105225


In [102]:
rrr = df.loc[df['patent_id'] == 10502806]
rrr['patent_title'].values[0]

'Magnetic field sensor'

In [68]:
# start = time.time()
# #response = ollama.embeddings(model="all-minilm", prompt=doc1)

# me_index = 0

# title_dict = {}

# #for row in smaller_df.itertuples(index=False):
# for row in df.itertuples(index=False):
# #for row in df.iterrows():
#     #print(row[smaller_df.columns.get_loc('patent_id')])
#     #response = ollama.embeddings(model="all-minilm", prompt=row[smaller_df.columns.get_loc('patent_title')])
#     #response = ollama.embeddings(model="nomic-embed-text", prompt=row[smaller_df.columns.get_loc('patent_title')])
#     #response = ollama.embeddings(model="mxbai-embed-large", prompt=row[smaller_df.columns.get_loc('patent_title')])
#     #response = ollama.embeddings(model="all-minilm", prompt=row[df.columns.get_loc('patent_title')])
#     #response = ollama.embeddings(model="nomic-embed-text", prompt=row[df.columns.get_loc('patent_title')])
#     #response = ollama.embeddings(model="mxbai-embed-large", prompt=row[df.columns.get_loc('patent_title')])
#     #response = ollama.embeddings(model="all-minilm", prompt=row[df.columns.get_loc('patent_abstract')])
#     #response = ollama.embeddings(model="nomic-embed-text", prompt=row[df.columns.get_loc('patent_abstract')])
#     response = ollama.embeddings(model="mxbai-embed-large", prompt=row[df.columns.get_loc('patent_abstract')])

#     # if me_index == -1:
#     #     val = list(response.values())[0]
#     #     #print(val)
#     #     print(len(val))
#     info = list(response.values())[0]
#     arr_info = np.array(info)
#     title_dict[row.patent_id] = arr_info
#     #print(row.patent_id)
#     me_index += 1

# end = time.time()
# dtime = "Execution time = " + str(end - start)
# print(dtime)

Execution time = 1192.223415851593
